In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

Path("figs").mkdir(exist_ok=True)

files = {"DCA": "faa_dca.xlsx", "BWI": "faa_bwi.xlsx"}
dfs = []
for code, path in files.items():
    df = pd.read_excel(path)
    # normalize headers (lowercase, strip, spaces->underscores)
    df.columns = df.columns.str.lower().str.strip().str.replace(r"\s+","_", regex=True)
    df["airport"] = code
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

# ---- dates ----
date_col = "incident_date" if "incident_date" in df.columns else "date"
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df = df.dropna(subset=[date_col])
df["year"]  = df[date_col].dt.year
df["month"] = df[date_col].dt.to_period("M")

# ---- damage (your files have damage_level + indicated_damage) ----
has_lvl  = "damage_level" in df.columns
has_bool = "indicated_damage" in df.columns

if not (has_lvl or has_bool):
    raise ValueError("No damage column found; re-download the full Excel (not the 8-column table).")

codes_damaged = {"m","m?","s","d","minor","substantial","destroyed"}  # anything other than 'N'

lvl = df["damage_level"].astype(str).str.lower() if has_lvl else pd.Series("", index=df.index)
damaging = lvl.isin(codes_damaged)
if has_bool:
    damaging = damaging | (df["indicated_damage"] == True)
df["damage_norm_any"] = damaging  # boolean you can group on

# ---- species column ----
species_col = "species" if "species" in df.columns else [c for c in df.columns if "species" in c][0]

# 1) monthly trend
from matplotlib.dates import YearLocator, DateFormatter

monthly = (df.groupby(["airport", df["incident_date"].dt.to_period("M")])
             .size().rename("strikes").reset_index())
pivot = monthly.pivot(index="incident_date", columns="airport", values="strikes").sort_index()
ts = pivot.index.to_timestamp()

ax = pivot.set_index(ts).rolling(3, min_periods=1).mean().plot(
        figsize=(9,4), title="Bird strikes over time (3-month rolling)")
ax.set_xlabel("Year")
ax.set_ylabel("Strikes")
ax.xaxis.set_major_locator(YearLocator(1))
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
plt.tight_layout(); plt.savefig("figs/monthly_trend_rolling.png", dpi=200); plt.close()


# 2) percent with any damage
rate = (df.groupby("airport")["damage_norm_any"].mean().mul(100).round(1)
          .rename("damage_rate_pct"))
ax = rate.plot(kind="bar", figsize=(6,4), title="Percent of strikes causing damage")
ax.set_ylabel("% with damage"); plt.xticks(rotation=0)
plt.tight_layout(); plt.savefig("figs/damage_rate_dca_bwi.png", dpi=200); plt.close()

# 3) top species in damaging strikes (combined airports)
top_species = (df[df["damage_norm_any"]]
               .groupby(species_col).size()
               .sort_values(ascending=False).head(10))
ax = top_species.plot(kind="barh", figsize=(8,5),
                      title="Top species in damaging strikes (DCA + BWI, 2010+)")
ax.set_xlabel("Damaging strikes")
plt.tight_layout(); plt.savefig("figs/top_species_damaging_dca_bwi.png", dpi=200); plt.close()

print("Saved figs in ./figs")

Saved figs in ./figs
